In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow.compat.v1 as tf
tf.disable_eager_execution()
tf.logging.set_verbosity(tf.logging.ERROR)

import warnings
warnings.filterwarnings('ignore')

from sklearn.compose import make_column_transformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import OneHotEncoder

from aif360.sklearn.preprocessing import Reweighing, ReweighingMeta, FairAdapt, LearnedFairRepresentations
from aif360.sklearn.inprocessing import AdversarialDebiasing, ExponentiatedGradientReduction, GridSearchReduction
from aif360.sklearn.postprocessing import CalibratedEqualizedOdds, RejectOptionClassifier, PostProcessingMeta, RejectOptionClassifierCV
from aif360.sklearn.datasets import fetch_adult
from aif360.sklearn.metrics import disparate_impact_ratio, average_odds_error, generalized_fpr
from aif360.sklearn.metrics import generalized_fnr, difference

# from glob import glob
# import json,os

2023-02-28 14:44:24.750914: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-28 14:44:24.931172: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-02-28 14:44:27.540738: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-02-28 14:44:27.540863: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] 

In [2]:
import json,os
from glob import glob
import traceback


def writejson(path,arr,name):
    try:
        os.mkdir(path+'preds/')
    except:
        pass
    f = open(path+'preds/'+name,'w')
    json.dump(arr.tolist(),f)
    f.close()

In [3]:
path = 'data/datasets/diabetes/'
synthfols = glob("data/datasets/diabetes/synthetic/*/")
paths = [path]
paths.extend(synthfols)

In [4]:
def task2(path):
    
    if os.path.exists(path+'preds/rocv_pred_average_odds.json') == False:
        print(path)

        train_df = pd.read_csv(path+'train.csv',header=None)

        train_df.columns = ['race', 'sex', 'age', 'admissiontypeid', 'dischargedispositionid',
       'admissionsourceid', 'timeinhospital', 'numlabprocedures',
       'numprocedures', 'nummedications', 'numberoutpatient',
       'numberemergency', 'numberinpatient', 'diag1', 'diag2', 'diag3',
       'numberdiagnoses', 'maxgluserum', 'A1Cresult', 'metformin',
       'glimepiride', 'glipizide', 'glyburide', 'pioglitazone',
       'rosiglitazone', 'insulin', 'change', 'diabetesMed', 'readmitted']

        train_df.index = train_df['sex']

        train_df = train_df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

        test_df = pd.read_csv(path+'test.csv',header=None)

        test_df.columns = ['race', 'sex', 'age', 'admissiontypeid', 'dischargedispositionid',
       'admissionsourceid', 'timeinhospital', 'numlabprocedures',
       'numprocedures', 'nummedications', 'numberoutpatient',
       'numberemergency', 'numberinpatient', 'diag1', 'diag2', 'diag3',
       'numberdiagnoses', 'maxgluserum', 'A1Cresult', 'metformin',
       'glimepiride', 'glipizide', 'glyburide', 'pioglitazone',
       'rosiglitazone', 'insulin', 'change', 'diabetesMed', 'readmitted']


        test_df.index = test_df['sex']

        test_df = test_df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)



        X_train = train_df.drop(['readmitted'],axis=1)
        X_test = test_df.drop(['readmitted'],axis=1)

        y_train = pd.Series(train_df['readmitted'])
        y_test = pd.Series(test_df['readmitted'])

        y_train = pd.Series(y_train.factorize(sort=True)[0], index=y_train.index)
        y_test = pd.Series(y_test.factorize(sort=True)[0], index=y_test.index)

        X_merged = pd.concat([X_train,X_test])

        ohe = make_column_transformer(
            (OneHotEncoder(sparse=False), X_merged.dtypes == 'object'),
            remainder='passthrough', verbose_feature_names_out=False)

        X_merged_temp  = pd.DataFrame(ohe.fit_transform(X_merged), columns=ohe.get_feature_names_out(), index=X_merged.index)

        X_train  = pd.DataFrame(ohe.transform(X_train), columns=ohe.get_feature_names_out(), index=X_train.index)
        X_test = pd.DataFrame(ohe.transform(X_test), columns=ohe.get_feature_names_out(), index=X_test.index)


        #### BASELINE
        try:
            y_pred_baseline = LogisticRegression(solver='liblinear').fit(X_train, y_train).predict(X_test)
            writejson(path,y_pred_baseline,'baseline_pred.json')    
        except:
            traceback.print_exc()


        #### Adv. Reweighting (Preproc.)

        try:
            rew = ReweighingMeta(estimator=LogisticRegression(solver='liblinear'),
                         reweigher=Reweighing('sex'))
            params = {'estimator__C': [1, 10]}

            clf = GridSearchCV(rew, params, scoring='accuracy', cv=5)
            clf.fit(X_train, y_train)
            y_pred_advrew = clf.predict(X_test)
            writejson(path,y_pred_advrew,'adv_rew_pred.json')  
        except:
            traceback.print_exc()



        #### LearnedFairRepresentations (Preproc.)

        try:
            LFR = LearnedFairRepresentations(prot_attr='sex')
            LFR.fit(X_train, y_train)

            y_pred_lfr = LFR.predict(X_test)
            writejson(path,y_pred_lfr,'lfr_pred.json')  
        except:
            traceback.print_exc()

        #### Adv Deb. (Inproc.)

        try:
            adv_deb = AdversarialDebiasing(prot_attr='sex')
            adv_deb.fit(X_train, y_train)
            adv_deb.score(X_test, y_test)

            y_pred_adv_deb = adv_deb.predict(X_test)
            writejson(path,y_pred_adv_deb,'adv_deb_pred.json')

            adv_deb.sess_.close() 
        except:
            traceback.print_exc()



        #### ExponentiatedGradientReduction. (Inproc.)

        try:
            consts = ['EqualizedOdds']#['DemographicParity','EqualizedOdds','TruePositiveRateParity','FalsePositiveRateParity','ErrorRateParity']
            for const in consts:
                EGR = ExponentiatedGradientReduction(prot_attr=['sex_Female','sex_Male'], estimator=LogisticRegression(solver='liblinear'), constraints = const)
                EGR.fit(X_train, y_train)
                print(const,EGR.score(X_test, y_test))
                y_pred_egr = EGR.predict(X_test)
                writejson(path,y_pred_egr,'egr_pred_'+const+'.json')
        except:
            traceback.print_exc()

        #### GridSearchReduction. (Inproc.)

        try:
            consts = ['EqualizedOdds']#['DemographicParity','EqualizedOdds','TruePositiveRateParity','FalsePositiveRateParity','ErrorRateParity']
            for const in consts:
                GSR = GridSearchReduction(prot_attr=['sex_Female','sex_Male'], estimator=LogisticRegression(solver='liblinear'), constraints = const)
                GSR.fit(X_train, y_train)
                print(const,GSR.score(X_test, y_test))
                y_pred_gsr = GSR.predict(X_test)
                writejson(path,y_pred_gsr,'gsr_pred_'+const+'.json')
        except:
            traceback.print_exc()



        X_train_temp = X_train.set_index(['sex_Male'], drop = False)
        y_train_temp = y_train.copy()
        y_train_temp.index = X_train_temp.index

        X_test_temp = X_test.set_index(['sex_Male'], drop = False)
        y_test_temp = y_test.copy()
        y_test_temp.index = X_test_temp.index

        #### Cal. Eq.Odds (Postproc.)

        try:
            consts = ['weighted']#['fnr','fpr','weighted']
            for const in consts:
                cal_eq_odds = CalibratedEqualizedOdds(prot_attr='sex_Male', cost_constraint=const)
                postproc = PostProcessingMeta(estimator=LogisticRegression(solver='liblinear'), postprocessor=cal_eq_odds)
                postproc.fit(X_train_temp, y_train_temp)
                print(accuracy_score(y_test_temp, postproc.predict(X_test_temp)))
                y_pred_caleq = postproc.predict(X_test_temp)
                writejson(path,y_pred_caleq,'caleq_pred_'+const+'.json')
        except:
            traceback.print_exc()

        #### RejectOptionClassifier (Postproc.)

        try:
            consts = ['average_odds']#['statistical_parity', 'average_odds', 'equal_opportunity']
            for const in consts:
                rocv = PostProcessingMeta(LogisticRegression(solver='liblinear'), RejectOptionClassifierCV('sex_Male', scoring=const))
                rocv.fit(X_train_temp, y_train_temp)
                print(accuracy_score(y_test_temp, rocv.predict(X_test_temp)))
                y_pred_rocv = rocv.predict(X_test_temp)
                writejson(path,y_pred_rocv,'rocv_pred_'+const+'.json')
        except:
            traceback.print_exc()  

        return path,"Done"


In [ ]:
for path in paths:
    try:
        task2(path)
    except:
        traceback.print_exc()

data/datasets/diabetes/


2023-02-28 14:44:47.618508: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-28 14:44:48.949921: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 46704 MB memory:  -> device: 0, name: NVIDIA RTX A6000, pci bus id: 0000:b1:00.0, compute capability: 8.6
2023-02-28 14:44:48.950749: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 46704 MB memory:  -> device: 1, name: NVIDIA RTX A6000, pci bus id: 0000:d9:00.0, compute capability: 8.6
2023-02-28 14:44:49.466127: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:357] MLIR V1 optimization pass is not e

EqualizedOdds 0.6252097315436241
EqualizedOdds 0.6198965324384788
0.6225531319910514
0.6180788590604027
data/datasets/diabetes/synthetic/gender_flip_labels0.1_version0/


2023-02-28 14:49:04.581001: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 46704 MB memory:  -> device: 0, name: NVIDIA RTX A6000, pci bus id: 0000:b1:00.0, compute capability: 8.6
2023-02-28 14:49:04.585246: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 46704 MB memory:  -> device: 1, name: NVIDIA RTX A6000, pci bus id: 0000:d9:00.0, compute capability: 8.6


EqualizedOdds 0.6312919463087249
EqualizedOdds 0.6305229306487695
0.6259787472035794
0.6205956375838926
data/datasets/diabetes/synthetic/gender_flip_labels0.1_version1/


...

2023-02-28 19:51:00.613043: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 46704 MB memory:  -> device: 0, name: NVIDIA RTX A6000, pci bus id: 0000:b1:00.0, compute capability: 8.6
2023-02-28 19:51:00.617786: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 46704 MB memory:  -> device: 1, name: NVIDIA RTX A6000, pci bus id: 0000:d9:00.0, compute capability: 8.6


EqualizedOdds 0.6270274049217002
EqualizedOdds 0.6273769574944071
0.6265380313199105
0.6208752796420581
data/datasets/diabetes/synthetic/gender_flip_labels0.5_version2/


2023-02-28 19:55:39.337956: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 46704 MB memory:  -> device: 0, name: NVIDIA RTX A6000, pci bus id: 0000:b1:00.0, compute capability: 8.6
2023-02-28 19:55:39.348109: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 46704 MB memory:  -> device: 1, name: NVIDIA RTX A6000, pci bus id: 0000:d9:00.0, compute capability: 8.6


EqualizedOdds 0.6233920581655481
EqualizedOdds 0.6255592841163311
0.6233920581655481
0.6199664429530202
data/datasets/diabetes/synthetic/gender_flip_labels0.5_version3/


2023-02-28 19:59:46.959714: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 46704 MB memory:  -> device: 0, name: NVIDIA RTX A6000, pci bus id: 0000:b1:00.0, compute capability: 8.6
2023-02-28 19:59:46.960192: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 46704 MB memory:  -> device: 1, name: NVIDIA RTX A6000, pci bus id: 0000:d9:00.0, compute capability: 8.6
